In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip -n -q "/content/drive/MyDrive/archive.zip" -d "/content/drive/MyDrive/"


In [ ]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.applications import EfficientNetB3, ResNet50, InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix

# Step 1: Data Loading

# Define directories
train_dir = '/content/drive/MyDrive/Data/train'
valid_dir = '/content/drive/MyDrive/Data/valid'
test_dir = '/content/drive/MyDrive/Data/test'

# Function to create data frames from image paths and labels
def create_dataframe(directory):
    labels = os.listdir(directory)
    file_paths = []
    categories = []
    for label in labels:
        label_dir = os.path.join(directory, label)
        for file in os.listdir(label_dir):
            file_paths.append(os.path.join(label_dir, file))
            categories.append(label)
    df = pd.DataFrame({'File_Path': file_paths, 'Category': categories})
    return df

# Create data frames for training, validation, and testing
train_df = create_dataframe(train_dir)
valid_df = create_dataframe(valid_dir)
test_df = create_dataframe(test_dir)

# Step 2: Image Data Generators

# Define parameters
image_size = (300, 300)
batch_size = 32

# Data augmentation for training set
train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True
)

# No augmentation for validation and test sets
valid_test_datagen = ImageDataGenerator(
    rescale=1./255
)

# Create data generators
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col='File_Path',
    y_col='Category',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

valid_generator = valid_test_datagen.flow_from_dataframe(
    valid_df,
    x_col='File_Path',
    y_col='Category',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = valid_test_datagen.flow_from_dataframe(
    test_df,
    x_col='File_Path',
    y_col='Category',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Step 3: Model Training (EfficientNetB3, ResNet50, InceptionV3)

# Define input shape
input_shape = (*image_size, 3)

# Load pre-trained models without top layer
base_model_efficientnet = EfficientNetB3(weights='imagenet', include_top=False, input_shape=input_shape)
base_model_resnet50 = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
base_model_inception = InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)

# Add custom classification layers
x = GlobalAveragePooling2D()(base_model_efficientnet.output)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions_efficientnet = Dense(len(train_df['Category'].unique()), activation='softmax')(x)
model_efficientnet = Model(inputs=base_model_efficientnet.input, outputs=predictions_efficientnet)

x = GlobalAveragePooling2D()(base_model_resnet50.output)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions_resnet50 = Dense(len(train_df['Category'].unique()), activation='softmax')(x)
model_resnet50 = Model(inputs=base_model_resnet50.input, outputs=predictions_resnet50)

x = GlobalAveragePooling2D()(base_model_inception.output)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions_inception = Dense(len(train_df['Category'].unique()), activation='softmax')(x)
model_inception = Model(inputs=base_model_inception.input, outputs=predictions_inception)



Found 613 validated image filenames belonging to 4 classes.
Found 72 validated image filenames belonging to 4 classes.
Found 315 validated image filenames belonging to 4 classes.
87910968/87910968 [==============================] - 0s 0us/step


In [ ]:
# Compile models
models = [model_efficientnet, model_resnet50, model_inception]
for model in models:
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train models
for model in models:
    checkpoint = ModelCheckpoint(filepath=model.name+'_best_model.h5', save_best_only=True, monitor='val_accuracy', mode='max')
    model.fit(
        train_generator,
        epochs=10,
        validation_data=valid_generator,
        callbacks=[checkpoint]
    )

# Load best versions of models
best_models = [Model(inputs=model.input, outputs=model.output) for model in models]
for i, model in enumerate(best_models):
    model.load_weights(models[i].name+'_best_model.h5')

# Create ensemble model
ensemble_inputs = [model.input for model in best_models]
ensemble_outputs = [model.output for model in best_models]
ensemble_output = Dense(len(train_df['Category'].unique()), activation='softmax')(concatenate(ensemble_outputs))
ensemble_model = Model(inputs=ensemble_inputs, outputs=ensemble_output)

# Compile ensemble model
ensemble_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


Epoch 1/10
20/20 [==============================] - ETA: 0s - loss: 0.8536 - accuracy: 0.6297

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


20/20 [==============================] - 198s 6s/step - loss: 0.8536 - accuracy: 0.6297 - val_loss: 1.7253 - val_accuracy: 0.2083
Epoch 2/10
20/20 [==============================] - 19s 931ms/step - loss: 0.4617 - accuracy: 0.8157 - val_loss: 3.7307 - val_accuracy: 0.2083
Epoch 3/10
20/20 [==============================] - 19s 958ms/step - loss: 0.2368 - accuracy: 0.9038 - val_loss: 3.0649 - val_accuracy: 0.2083
Epoch 4/10
20/20 [==============================] - 21s 1s/step - loss: 0.3401 - accuracy: 0.8940 - val_loss: 2.3374 - val_accuracy: 0.2917
Epoch 5/10
20/20 [==============================] - 20s 982ms/step - loss: 0.2419 - accuracy: 0.9135 - val_loss: 16.5372 - val_accuracy: 0.1806
Epoch 6/10
20/20 [==============================] - 20s 1s/step - loss: 0.2985 - accuracy: 0.9168 - val_loss: 4.7861 - val_accuracy: 0.1806
Epoch 7/10
20/20 [==============================] - 19s 945ms/step - loss: 0.2708 - accuracy: 0.9184 - val_loss: 9.8545 - val_accuracy: 0.1806
Epoch 8/10
20/20 

In [ ]:
# Generate predictions for the test data using the ensemble model
test_generator.reset()  # Reset generator to start from the beginning
y_pred_list = []
for i in range(len(test_generator)):
    batch_images, _ = test_generator[i]
    y_pred_batch = ensemble_model.predict([batch_images] * len(best_models))
    y_pred_list.append(y_pred_batch)

# Concatenate predictions from all batches
y_pred = np.concatenate(y_pred_list)

# Get true labels from the test generator
y_true = []
for i in range(len(test_generator)):
    _, labels = test_generator[i]
    y_true.extend(np.argmax(labels, axis=1))

# Convert predicted probabilities to class labels
y_pred_labels = np.argmax(y_pred, axis=1)

# Compute classification report
report = classification_report(y_true, y_pred_labels)

# Print the classification report
print(report)


1/1 [==============================] - 16s 16s/step
              precision    recall  f1-score   support

           0       0.51      0.60      0.55       120
           1       0.00      0.00      0.00        51
           2       0.00      0.00      0.00        54
           3       0.23      0.43      0.30        90

    accuracy                           0.35       315
   macro avg       0.18      0.26      0.21       315
weighted avg       0.26      0.35      0.29       315



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
